In [ ]:
%matplotlib inline

from copy import deepcopy
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime


In [ ]:
%matplotlib inline

# 1) Chargement de la machine de référence
ref_path = join(DATA_DIR, "Machine", "Tesla_model_3.json")
M_ref = load(ref_path)

print("=" * 80)
print("GÉNÉRATION DATASET DÉFAUTS USINAGE - TESLA MODEL 3")
print("=" * 80)

# 2) Analyse de la machine de référence
print("\n ANALYSE DE LA MACHINE DE RÉFÉRENCE")
print("=" * 45)

print(f"Type de machine: {type(M_ref).__name__}")
print(f"Nom: {M_ref.name}")

# Analyse des encoches stator
if hasattr(M_ref.stator, 'slot'):
    slot = M_ref.stator.slot
    print(f"Type d'encoche: {type(slot).__name__}")
    
    if hasattr(slot, 'W0'):
        print(f"Largeur ouverture (W0): {slot.W0*1000:.2f} mm")
    if hasattr(slot, 'H0'):
        print(f"Hauteur isthme (H0): {slot.H0*1000:.2f} mm")
    if hasattr(slot, 'W1'):
        print(f"Largeur sommet (W1): {slot.W1*1000:.2f} mm")
    if hasattr(slot, 'H1'):
        print(f"Hauteur coin (H1): {slot.H1*1000:.2f} mm")
    if hasattr(slot, 'W2'):
        print(f"Largeur base (W2): {slot.W2*1000:.2f} mm")
    if hasattr(slot, 'H2'):
        print(f"Hauteur encoche (H2): {slot.H2*1000:.2f} mm")

# 3) Fonction de validation pour défauts usinage
def validate_machining_machine(machine, name):
    """Validation spécifique pour machines avec défauts usinage"""
    errors = []
    warnings = []
    
    # Vérification entrefer
    airgap = machine.stator.Rint - machine.rotor.Rext
    if airgap <= 0:
        errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
    elif airgap < 0.3e-3:
        warnings.append(f"Entrefer très petit: {airgap*1000:.2f} mm")
    
    # Vérification bobinage
    if hasattr(machine.stator.winding, 'Ntcoil'):
        if machine.stator.winding.Ntcoil < 5:
            errors.append(f"Tours par bobine trop faibles: {machine.stator.winding.Ntcoil}")
    
    # Vérification des dimensions d'encoches
    if hasattr(machine.stator, 'slot'):
        slot = machine.stator.slot
        if hasattr(slot, 'W0') and slot.W0 <= 0:
            errors.append(f"Largeur ouverture invalide: {slot.W0*1000:.2f} mm")
        if hasattr(slot, 'H0') and slot.H0 <= 0:
            errors.append(f"Hauteur isthme invalide: {slot.H0*1000:.2f} mm")
        if hasattr(slot, 'W1') and slot.W1 <= 0:
            errors.append(f"Largeur sommet invalide: {slot.W1*1000:.2f} mm")
        if hasattr(slot, 'H1') and slot.H1 <= 0:
            errors.append(f"Hauteur coin invalide: {slot.H1*1000:.2f} mm")
    
    return errors, warnings

# 4) Génération des machines avec défauts usinage
print("\n GÉNÉRATION DES MACHINES DÉFAUTS USINAGE")
print("=" * 50)

# on definie ici les niveaux de défaut usinage (-10% à +10%)
machining_variations = np.linspace(-0.10, 0.10, 21)  # 21 niveaux de -10% à +10%

print(f" Niveaux de défaut usinage: {machining_variations*100}%")
print(f" Nombre de machines: {len(machining_variations)}")

machines = []
names = []

for i, mach_level in enumerate(machining_variations):
    print(f"\n--- Génération machine {i+1:2d}/{len(machining_variations)} ---")
    
    # Copier la machine de référence
    M = deepcopy(M_ref)
    
    # on applique le défaut usinage sur les encoches stator
    if hasattr(M.stator, 'slot'):
        slot = M.stator.slot
        
        # Save des valeurs originales
        original_values = {}
        modified_values = {}
        
        # Modification de W0 (largeur ouverture)
        if hasattr(slot, 'W0'):
            original_values['W0'] = slot.W0
            slot.W0 *= (1 + mach_level)
            modified_values['W0'] = slot.W0
        
        # Modification de H0 (hauteur isthme)
        if hasattr(slot, 'H0'):
            original_values['H0'] = slot.H0
            slot.H0 *= (1 + mach_level)
            modified_values['H0'] = slot.H0
        
        # Modification de  W1 (largeur sommet)
        if hasattr(slot, 'W1'):
            original_values['W1'] = slot.W1
            slot.W1 *= (1 + mach_level)
            modified_values['W1'] = slot.W1
        
        # Modification de  H1 (hauteur coin)
        if hasattr(slot, 'H1'):
            original_values['H1'] = slot.H1
            slot.H1 *= (1 + mach_level)
            modified_values['H1'] = slot.H1
        
        # Modification de  W2 (largeur base)
        if hasattr(slot, 'W2'):
            original_values['W2'] = slot.W2
            slot.W2 *= (1 + mach_level)
            modified_values['W2'] = slot.W2
        
        # Modification de  H2 (hauteur encoche)
        if hasattr(slot, 'H2'):
            original_values['H2'] = slot.H2
            slot.H2 *= (1 + mach_level)
            modified_values['H2'] = slot.H2
        
        # affichage des modifications
        print(f"   Défaut usinage: {mach_level*100:+.1f}%")
        for param, orig_val in original_values.items():
            new_val = modified_values[param]
            print(f"   {param}: {orig_val*1000:.2f} mm → {new_val*1000:.2f} mm")
    
    # machine name
    sign = "Plus" if mach_level > 0 else "Moins" if mach_level < 0 else "Zero"
    machine_name = f"Tesla_Model3_Defaut_Usinage_{sign}{abs(mach_level)*100:.0f}_Pourcent"
    
    machines.append(M)
    names.append(machine_name)
    
    print(f" {machine_name}")

# 5) Validation des machines générées
print(f"\n VALIDATION DES MACHINES GÉNÉRÉES")
print("=" * 45)

valid_machines = []
valid_names = []

for Mi, nm in zip(machines, names):
    print(f"\n--- Validation de {nm} ---")
    
    # Validation spécifique
    errors, warnings = validate_machining_machine(Mi, nm)
    
    if errors:
        print(f" ERREURS critiques:")
        for error in errors:
            print(f"   - {error}")
        print(f"   → Machine rejetée")
        continue
    
    if warnings:
        print(f" Avertissements:")
        for warning in warnings:
            print(f"   - {warning}")
    
    print(f" {nm}: Validation réussie")
    valid_machines.append(Mi)
    valid_names.append(nm)

# 6) Sauvegarde du dataset défauts usinage
print(f"\n SAUVEGARDE DU DATASET DÉFAUTS USINAGE")
print("=" * 50)

# Création du dossier
save_dir = "Tesla_Model3_Defauts_Usinage"
os.makedirs(save_dir, exist_ok=True)

print(f" Dossier créé: {os.path.abspath(save_dir)}/")

# Sauvegarde de la machine de référence
ref_path = os.path.join(save_dir, "Tesla_Model3_Reference.json")
M_ref.save(ref_path)
print(f" Machine de référence sauvegardée")

# Sauvegarde des machines avec défauts usinage
if valid_machines:
    print(f"\n SAUVEGARDE DES MACHINES VALIDES")
    print("=" * 40)
    
    for Mi, nm in zip(valid_machines, valid_names):
        out_path = os.path.join(save_dir, nm + ".json")
        Mi.save(out_path)
        print(f" Sauvegardé: {nm}")
        
        # Affichage de la machine
        fig, ax = Mi.plot(is_show_fig=False)
        plt.title(nm, fontsize=10)
        plt.show()
    
    # Rapport de sauvegarde
    print(f"\n RAPPORT DE SAUVEGARDE")
    print("=" * 30)
    print(f" Total machines sauvegardées: {len(valid_names) + 1}")
    print(f"   - Machine de référence: 1")
    print(f"   - Machines défauts usinage: {len(valid_names)}")
    print(f"   - Plage de défaut usinage: -10% à +10%")
    
    # Vérification des fichiers créés
    print(f"\n VÉRIFICATION DES FICHIERS")
    print("=" * 30)
    
    total_files = 0
    total_size = 0
    for root, dirs, files in os.walk(save_dir):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                total_files += 1
                total_size += file_size
                print(f" {file} - {file_size:,} octets")
    
    print(f"\n SUCCÈS: {total_files} fichiers JSON créés")
    print(f" Taille totale: {total_size:,} octets ({total_size/1024:.1f} KB)")
    
    # Affichage du chemin complet
    print(f"\n EMPLACEMENT FINAL:")
    print(f"   {os.path.abspath(save_dir)}/")
    
    # Liste des machines générées
    print(f"\n LISTE DES MACHINES GÉNÉRÉES:")
    print("=" * 35)
    print(f"1. Tesla_Model3_Reference.json")
    for i, name in enumerate(valid_names, 2):
        print(f"{i:2d}. {name}.json")
    
    # Statistiques des variations
    print(f"\n STATISTIQUES DES VARIATIONS:")
    print("=" * 35)
    print(f"   Variations négatives: {sum(1 for v in machining_variations if v < 0)}")
    print(f"   Variation zéro: {sum(1 for v in machining_variations if v == 0)}")
    print(f"   Variations positives: {sum(1 for v in machining_variations if v > 0)}")
    print(f"   Plage totale: {machining_variations[0]*100:.1f}% à {machining_variations[-1]*100:.1f}%")
    
else:
    print(" Aucune machine valide générée")

print("\n" + "=" * 80)
print("GÉNÉRATION DU DATASET DÉFAUTS USINAGE TERMINÉE")
print("=" * 80)

In [ ]:
----------------------------------------------------------------------------------------------------------